In [31]:
# Primero cargamos las imágenes procesadas:

import numpy as np
import h5py

hf = h5py.File("Preprocesado/processed.h5", "r")
processed_images = np.array(hf.get("processed_images"))

In [32]:
# Si queremos hacer Data Augmentation, tendremos que cargar las imágenes sin estandarizar, únicamente recortadas.

import cv2 as cv

files = os.listdir("Preprocesado/Heridas/Recortados")
images = [cv.imread("Preprocesado/Heridas/Recortados/"+str(i+1)+".png") for i in range(len(files))]
sizes = [cv.imread("Preprocesado/Heridas/Recortados/"+str(i+1)+".png").shape[0] for i in range(len(files))]

# Ya tenemos una lista de las imágenes, ahora haremos Data Augmentation.

In [33]:
# Ahora haremos algo de DataAugmentation para conseguir algunas imágenes extra.

from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Consideraremos, como primera medida, obtener 4 imágenes modificadas, aleatoriamente, de cada una de las que tenemos realizando diferentes operaciones como
# traslaciones, rotaciones, zooms, ...
# Crearemos una nueva función de preprocesado específica para este caso.

def preprocessing_dataug(path_inj, path_healthy, name_img, img):
    import numpy as np
    import cv2 as cv

    img_inj = img
    img_healthy = cv.imread(path_healthy+"/"+name_img+".png")

    def standardize(img1, img2):
        mean = np.mean(img1)
        std = np.std(img1)
        img = (img2-mean)/std
        return img

    r_channel = standardize(img_healthy[:,:,0], img_inj[:,:,0])
    g_channel = standardize(img_healthy[:,:,1], img_inj[:,:,1])
    b_channel = standardize(img_healthy[:,:,2], img_inj[:,:,2])

    # Para evitar valores negativos en las imágenes de salida, aplicaremos la "unity-based normalization" tras estandarizar los canales.

    image_std = np.stack([r_channel, g_channel, b_channel], axis=-1)
    image_std_normalized = (image_std-np.min(image_std))/(np.max(image_std)-np.min(image_std))

    return image_std_normalized

In [34]:
new_images = []

for i in range(len(images)):

    dim_img = images[i].shape[0]
    samples = expand_dims(images[1], 0)
    datagen = ImageDataGenerator(width_shift_range = 0.3, height_shift_range = 0.3, horizontal_flip = True, rotation_range = 90, zoom_range = [0.5,1.0])
    it = datagen.flow(samples, batch_size = 1)

    for j in range(6):
        batch = it.next()
        image = batch[0].astype('uint8')
        image_new = preprocessing_dataug("Preprocesado/Heridas/Recortados", "Preprocesado/Sanos/Recortados", str(i+1), image)
        new_images.append(image_new)

In [35]:
# Ahora hemos de preprocesar también las imágenes originales y luego concatenar ambas listas (la original y la creada)

for i in range(len(files)):
    images[i] = preprocessing_dataug("Preprocesado/Heridas/Recortados", "Preprocesado/Sanos/Recortados", str(i+1), images[i])

data_augmented = images+new_images

In [36]:
# Ahora haremos un resize y las dispondremos en un array. Luego extraeremos el resultaod a un archivo h5.

from Funciones import resize

new_dim = int(round(np.mean(sizes)))

new_data_augmented = [resize(data_augmented[i], X = new_dim, Y = new_dim, method = "linear") for i in range(len(data_augmented))]

new_data_augmented = np.stack(new_data_augmented, axis = 0)

In [37]:
# Ahora hemos de sacar las etiquetas del CSV.

import pandas as pd

data = pd.read_csv("Leucoplasia307.csv")

labels = np.array(data["Evolucioncancer"].values)
milabel = np.array(data["MiLabel"].values)

indices = np.where(milabel > 0)[0]
labels = list(labels[indices])

# No obstante estas etiquetas son únicamente las de las imágenes originales y necesitamos etiquetar también las obtenidas mediante Data Augmentation.

for i in range(processed_images.shape[0]):
    labels = labels + [labels[i], labels[i], labels[i], labels[i], labels[i], labels[i]]

labels = np.array(labels)

In [38]:
# Del mismo CSV también queremos extraer la histologia.
import pandas as pd

histologia = np.array(data["Histologia"].values)
histologia = list(histologia[indices])

for i in range(processed_images.shape[0]):
    histologia = histologia + [histologia[i], histologia[i], histologia[i], histologia[i], histologia[i], histologia[i]]

histologia = np.array(histologia)
histologia = pd.get_dummies(histologia)
histologia.index = range(1,85)

In [39]:
# Lo almacenaremos todo en un h5 para transportarlo.

hf = h5py.File("data_augmented.h5", "w")
hf.create_dataset("data_augmented", data = new_data_augmented)
hf.create_dataset("labels", data = labels)
hf.create_dataset("histologia", data = histologia)
hf.close()

In [40]:
histologia

,Displasia leve,Displasia moderada,Displasia severa,Sin displasia
1,0,0,0,1
2,0,1,0,0
3,0,0,0,1
4,0,0,0,1
5,0,0,0,1
...,...,...,...,...
80,0,0,1,0
81,0,0,1,0
82,0,0,1,0
83,0,0,1,0
